# Playground #2 - building Gen AI application

In [ ]:
# Dependencies installation
!pip install langchain langchain-community langchain-openai tiktoken langfuse lancedb langgraph docling langchain-docling gradio python-dotenv

In [5]:
import os
from google.colab import userdata
# Set environment variables from secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_HOST"] = userdata.get("LANGFUSE_HOST")

## 1. Knowledge base

In [ ]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from docling_core.transforms.chunker.hierarchical_chunker import HierarchicalChunker
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions, EasyOcrOptions


# Document to be ingested
SOURCE = "https://web.seducoahuila.gob.mx/biblioweb/upload/the_wonderful_wizard_of_oz.pdf"


# Configure converter
docling_converter = DocumentConverter(
    allowed_formats=[InputFormat.PDF],
    format_options={
        InputFormat.PDF: PdfFormatOption(
            pipeline_options=PdfPipelineOptions(
                do_ocr=False
            ),
            ocr_options=EasyOcrOptions(
                force_full_page_ocr=False,
                lang=["en"]
            )
        )
    }
)


# Configure loader
loader = DoclingLoader(
    file_path=SOURCE,
    export_type=ExportType.DOC_CHUNKS,
    chunker=HierarchicalChunker()
)

documents = loader.load_and_split()



In [ ]:
# play around exploring the content od documents collection

print(f"Number of text chunks: {len(documents)}")
print(f"First chunk: {documents[0].page_content}")
print(f"Metadata: {documents[0].metadata}")


In [21]:
# create knowledge base for lookups (LanceDB)

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import LanceDB
import lancedb


def safe_get_page_no(data):
  """Helper function that extracts page number from metadata """
  dl_meta = data.get('dl_meta', {})
  doc_items = dl_meta.get('doc_items', [])
  if not doc_items: return None
  prov = doc_items[0].get('prov', [])
  return prov[0].get('page_no') if prov else None


embedding_function = OpenAIEmbeddings()

vector_store = LanceDB.from_texts(
    [doc.page_content.replace("\t", " ") for doc in documents],
    embedding_function,
    metadatas=[{"page_no": safe_get_page_no(doc.metadata)} for doc in documents],
    table_name="knowledge_base",
    connection=lancedb.connect("temp/wizard_of_oz")
)

In [ ]:
# play around with vector_store

vector_store.similarity_search("How many witches where in the book?", k=5)


In [ ]:
vector_store.similarity_search_with_relevance_scores("Who was the tinman?", k=4)

# 2. Orchestrating the application flow

In [25]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI

NUMBER_OF_SNIPPETS_TO_RETRIEVE = 5

# Define graph state
class RAGState(TypedDict):
    question: str
    snippets: list
    answer: str


# Node #1 - retrieval
def retrieve_snippets(state: RAGState):
  retriever = vector_store.as_retriever(search_kwargs={"k": NUMBER_OF_SNIPPETS_TO_RETRIEVE})
  snippets = retriever.get_relevant_documents(state["question"])
  return {"snippets": snippets}


# Node #2 - generation
def generate_answer(state: RAGState):
  context = "\n\n".join(doc.page_content for doc in state["snippets"])
  prompt = f"Using the context below, answer the question.\n\nQuestion: {state['question']}\nContext: {context}\nAnswer:"
  answer = ChatOpenAI(temperature=0.5, model_name="gpt-4o-mini").invoke(prompt).content
  return {"answer": answer}

# Build the state graph with sequential nodes
graph_builder = StateGraph(RAGState).add_sequence([retrieve_snippets, generate_answer])
graph_builder.add_edge(START, "retrieve_snippets")
graph_builder.add_edge("generate_answer", END)
graph = graph_builder.compile()


In [ ]:
# Try running the RAG generation
response = graph.invoke({'question': "What shoes did Dorothy wear?"})
response['answer']

In [ ]:
# Explore the response object
response

# 3. Adding a UI

In [ ]:
import gradio as gr

def answer_oz_question(question: str):
    response = graph.invoke({'question': question})
    return response['answer']

demo = gr.Interface(
    fn=answer_oz_question,
    inputs=gr.Textbox(label="Question"),
    outputs=gr.TextArea(label="Answer", interactive=False)
)
demo.launch(share=True)

* exercise 1: Try to make number of snippets used for answer generation configurable via the UI
* exercise 2*: Try to add the information which page was used for the answer generation a part of the answer